## 1. Import libraries and dependencies

In [1]:
from src.preprocess.preprocess import Preprocess
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import time
from src.preprocess.pipeline import Pipeline

2024-06-01 19:19:56.038205: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-01 19:20:01.288986: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## 2. Preprocess audio

In [2]:
ORIGIN_SAMPLE_RATE = 48_000
FINAL_SAMPLE_RATE = 16_000
MAX_CLIENT_ID_AMOUNT = 2500
MIN_CLIP_DURATION_MS = 6000
SET_SIZE = 18_000
BATCH_SIZE = 64

In [3]:
Preprocess.initialize(
    batch_size=BATCH_SIZE,
    max_client_id_amount=MAX_CLIENT_ID_AMOUNT,
    min_clip_duration_ms=MIN_CLIP_DURATION_MS,
    set_size=SET_SIZE,
    origin_sample_rate=ORIGIN_SAMPLE_RATE,
    final_sample_rate=FINAL_SAMPLE_RATE,
)

In [4]:
print(f"Original sample rate: {Preprocess.ORIGIN_SAMPLE_RATE}")
print(f"Final sample rate: {Preprocess.SAMPLE_RATE}")
print(f"Max client id amount: {Preprocess.MAX_CLIENT_ID_AMOUNT}")
print(f"Min clip duration in milliseconds: {Preprocess.MIN_CLIP_DURATION_MS}")
print(f"Set size: {Preprocess.SET_SIZE}")
print(f"One language set size: {Preprocess.ONE_LANG_SET_SIZE}")

print("-"*25 + "SETS SIZES" + "-"*25)
print(f"Training set size: {Preprocess.SET_HPARAMS.train_size}")
print(f"Val set size: {Preprocess.SET_HPARAMS.val_size}")
print(f"Test set size: {Preprocess.SET_HPARAMS.test_size}")
print(f"Is test + val + train is equal one lang set size: "
      f"\n{Preprocess.SET_HPARAMS.test_size + Preprocess.SET_HPARAMS.val_size + Preprocess.SET_HPARAMS.train_size} <= {Preprocess.ONE_LANG_SET_SIZE}")

Original sample rate: 48000
Final sample rate: 16000
Max client id amount: 2500
Min clip duration in milliseconds: 6000
Set size: 18000
One language set size: 6000
-------------------------SETS SIZES-------------------------
Training set size: 3600
Val set size: 1200
Test set size: 1200
Is test + val + train is equal one lang set size: 
6000 <= 6000


In [5]:
start = time.time()
Preprocess.preprocess_probes(save=True)
end = time.time()
print(f'Time taken: {end - start}')

PROCESSED LANGUAGES: de
---AUDIO META INFO DONE de---
---SET SPLIT DONE 0 de---
de 0 missing probes: 0
---SET SPLIT DONE 1 de---
de 1 missing probes: 0
PROCESSED LANGUAGES: fr
---AUDIO META INFO DONE fr---
---SET SPLIT DONE 0 fr---
fr 0 missing probes: 0
---SET SPLIT DONE 1 fr---
fr 1 missing probes: 0
PROCESSED LANGUAGES: pl
---AUDIO META INFO DONE pl---
---SET SPLIT DONE 0 pl---
pl 0 missing probes: 0
---SET SPLIT DONE 1 pl---
pl 1 missing probes: 51
Time taken: 57.57263708114624


In [6]:
from src.hyperparams.paths_info import PathsInfo
PathsInfo.LANG_DIR_PATH

'../../../UM-Project/languages'

In [7]:
import random
train = Preprocess.load_set("train")
val = Preprocess.load_set("val")
test = Preprocess.load_set("test")
random.shuffle(train)
random.shuffle(val)
random.shuffle(test)

In [8]:
# import random
# train = Preprocess.TRAIN_FILENAMES
# val = Preprocess.TEST_FILENAMES
# test = Preprocess.VAL_FILENAMES
# random.shuffle(train)
# random.shuffle(val)
# random.shuffle(test)

In [9]:
len(train), len(val), len(test)

(10800, 3600, 3600)

In [10]:
train_dataset = Pipeline.create_pipeline(train)
val_dataset = Pipeline.create_pipeline(val)
test_dataset = Pipeline.create_pipeline(test)

2024-06-01 19:21:26.076649: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-01 19:21:27.793248: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-06-01 19:21:28.176676: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA


In [12]:

# counter = 0
# start = time.time()
# for batch, label in train_dataset:
#     print(batch.shape)
# 
# end = time.time()
# print(f"Train it time: {end - start}")
# print(counter)


In [13]:
# counter = 0
# start = time.time()
# for batch, label in val_dataset:
#     print(batch.shape)
#     for sample in batch:
#         print(sample)
#         counter += 1
# end = time.time()
# print(f"Val it time: {end - start}")
# print(counter)

In [14]:
# counter = 0
# start = time.time()
# for batch, label in test_dataset:
#     print(batch.shape)
#     for sample in batch:
#         print(sample)
#         counter += 1
# end = time.time()
# print(f"Test it time: {end - start}")
# print(counter)

In [15]:
test_dataset.element_spec

(TensorSpec(shape=(64, 375, 256, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(64, 3), dtype=tf.int32, name=None))

In [16]:
val_dataset.element_spec

(TensorSpec(shape=(64, 375, 256, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(64, 3), dtype=tf.int32, name=None))

In [17]:
train_dataset.element_spec

(TensorSpec(shape=(64, 375, 256, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(64, 3), dtype=tf.int32, name=None))

## 4. Create model

In [18]:
input_shape = test_dataset.as_numpy_iterator().next()[0].shape[1:]
input_shape

2024-06-01 19:21:44.855880: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


(375, 256, 1)

In [19]:
model = keras.Sequential([
    layers.Input(shape=input_shape),
    layers.Normalization(axis=-1),
    layers.Conv2D(16, (7, 5), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((3, 2)),
    layers.Conv2D(32, (5, 5), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 375, 256, 1)    │             3 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 375, 256, 16)   │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 375, 256, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 125, 128, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 128, 32)   │        12,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 125, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 62, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 62, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 31, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 31, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 31, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 15, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 15, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 15, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 14336)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     3,670,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,074,982 (15.54 MB)

 Trainable params: 4,073,475 (15.54 MB)

 Non-trainable params: 1,507 (5.89 KB)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    'model3.keras',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=6
)

In [ ]:
history = model.fit(train_dataset, epochs=30, validation_data=val_dataset, batch_size=BATCH_SIZE, callbacks=[early_stopping, checkpoint_callback])

In [ ]:
# test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
# print(f'\nTest accuracy: {test_acc}')

In [ ]:
# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')
# plt.show()


In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model('model3.keras')

test_loss, test_acc = best_model.evaluate(test_dataset, verbose=2)
print(f'\nTest accuracy: {test_acc}')